In [ ]:
# === Registro robusto Arduino → CSV, con manejo de archivo bloqueado por Excel ===
# Requiere: pip install pyserial

import time
from datetime import datetime
from pathlib import Path
import serial
from serial.tools import list_ports

BAUD   = 9600
FNAME  = Path("dht_log.csv")
FLUSH_EVERY_SEC = 2.0     # intentar volcar el buffer cada 2 s si estuvo bloqueado

def detectar_puerto():
    """Intenta encontrar el puerto del Arduino por descripción/VID."""
    candidatos = []
    for p in list_ports.comports():
        desc = (p.description or "").lower()
        hwid = (p.hwid or "").lower()
        if any(k in desc for k in ["arduino", "uno", "mega", "ch340", "cp210"]):
            candidatos.append(p.device)
        elif any(v in hwid for v in ["vid_2341", "vid_2a03", "vid_1a86", "vid_10c4"]):
            candidatos.append(p.device)
    if candidatos:
        return candidatos[0]
    all_ports = [p.device for p in list_ports.comports()]
    return all_ports[0] if all_ports else None

def escribir_linea_segura(texto: str, buffer: list, last_try_ts: list):
    """
    Escribe en CSV. Si el archivo está bloqueado (Excel), guarda en buffer.
    last_try_ts es [timestamp] para poder modificarlo por referencia.
    """
    # Intenta escribir primero lo pendiente en buffer
    if buffer and (time.time() - last_try_ts[0] >= FLUSH_EVERY_SEC):
        try:
            with FNAME.open("a", encoding="utf-8") as f:
                f.writelines(buffer)
            buffer.clear()
        except PermissionError:
            # sigue bloqueado: pospone otro intento
            last_try_ts[0] = time.time()

    # Intenta escribir la línea actual
    try:
        with FNAME.open("a", encoding="utf-8") as f:
            f.write(texto)
    except PermissionError:
        # archivo bloqueado -> envía al buffer y avisa una vez cada cierto tiempo
        buffer.append(texto)
        if time.time() - last_try_ts[0] >= FLUSH_EVERY_SEC:
            print("⚠ No se pudo escribir (archivo abierto en Excel). Se acumula en buffer...")
            last_try_ts[0] = time.time()

def main():
    port = detectar_puerto()
    if not port:
        print("❌ No se encontró ningún puerto serie. Conecta el Arduino y vuelve a intentar.")
        return

    print(f"Intentando abrir {port}...")
    try:
        ser = serial.Serial(port, BAUD, timeout=2)
    except serial.SerialException as e:
        print(f"❌ No se pudo abrir {port}. ¿Está ocupado? Cierra el Monitor Serie. Detalle: {e}")
        return

    time.sleep(2)
    print(f"✅ Conectado a {port} @ {BAUD}. Guardando en {FNAME.name}")

    # Crear encabezado si el archivo no existe
    if not FNAME.exists():
        with FNAME.open("w", encoding="utf-8") as f:
            f.write("timestamp,time_ms,humidity_pct,temp_c\n")

    buffer = []             # líneas pendientes si Excel bloquea el archivo
    last_try_ts = [0.0]     # para control del reintento periódico

    try:
        while True:
            line = ser.readline().decode(errors="ignore").strip()
            if not line:
                continue

            parts = [p.strip() for p in line.split(",")]
            if len(parts) != 3:
                # Formato inesperado: ignora
                continue

            ts = datetime.now().isoformat(sep=" ", timespec="seconds")
            csv_line = f"{ts},{parts[0]},{parts[1]},{parts[2]}\n"

            # Escribe con tolerancia a archivo bloqueado
            escribir_linea_segura(csv_line, buffer, last_try_ts)

            # Muestra en consola
            print(ts, parts)

    except KeyboardInterrupt:
        print("\n🛑 Registro detenido por el usuario.")
    finally:
        ser.close()
        # Intento final de vaciar buffer
        if buffer:
            try:
                with FNAME.open("a", encoding="utf-8") as f:
                    f.writelines(buffer)
                print(f"📝 Se volcaron {len(buffer)} líneas pendientes al CSV.")
            except PermissionError:
                print(f"⚠ Quedaron {len(buffer)} líneas en buffer porque el archivo sigue bloqueado.")
        print("🔒 Puerto cerrado. Archivo guardado.")

if __name__ == "__main__":
    main()



Intentando abrir COM6...
✅ Conectado a COM6 @ 9600. Guardando en dht_log.csv
⚠ No se pudo escribir (archivo abierto en Excel). Se acumula en buffer...
2025-08-11 10:09:59 ['2021', '72.00', '27.80']
2025-08-11 10:10:01 ['4044', '71.00', '27.70']
2025-08-11 10:10:03 ['6068', '71.00', '27.50']
2025-08-11 10:10:05 ['8090', '71.00', '27.40']
2025-08-11 10:10:07 ['10114', '70.00', '27.50']
2025-08-11 10:10:09 ['12137', '70.00', '27.60']
2025-08-11 10:10:11 ['14161', '71.00', '27.80']
⚠ No se pudo escribir (archivo abierto en Excel). Se acumula en buffer...
2025-08-11 10:10:14 ['16184', '71.00', '27.80']
2025-08-11 10:10:16 ['18207', '71.00', '27.30']
2025-08-11 10:10:18 ['20232', '72.00', '27.50']
2025-08-11 10:10:20 ['22254', '72.00', '27.20']
2025-08-11 10:10:22 ['24278', '72.00', '27.70']
2025-08-11 10:10:24 ['26302', '72.00', '27.80']
2025-08-11 10:10:26 ['28324', '72.00', '27.70']
2025-08-11 10:10:28 ['30348', '73.00', '27.80']
2025-08-11 10:10:30 ['32371', '73.00', '27.60']
2025-08-11 